In [2]:
import requests
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


In [5]:
"""Initializing Spark Session"""
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
"""Read API That Shows University Information"""
r = requests.request("GET",'http://universities.hipolabs.com/search?country=United+States')
"""Print JSON Fetched file as python dictionary"""
data_j = r.json()
"""Place a list of JSON Objects into a spark dataframe"""
df = sc.parallelize(data_j).map(lambda x: json.dumps(x))
df = spark.read.json(df)
"""The File had a multitude of Null values in the state column which is what I wanted to work with"""
df = df.na.drop(subset=['state-province'])

"""Show Data-Set"""
print('Count of colleges:',df.count())
df.select('domains','name','state-province','web_pages').show()


Count of colleges: 83
+--------------------+--------------------+--------------+--------------------+
|             domains|                name|state-province|           web_pages|
+--------------------+--------------------+--------------+--------------------+
|[upmc.edu, upmc.com]|University of Pit...|  Pennsylvania|[https://www.upmc...|
|         [utrgv.edu]|The University of...|         Texas|[https://www.utrg...|
|           [uvu.edu]|Utah Valley Unive...|          Utah|  [https://uvu.edu/]|
|           [csn.edu]|College of Southe...|            NV|[https://www.csn....|
|        [bloomu.edu]|Bloomsburg Univer...|  Pennsylvania|[http://www.bloom...|
|           [cup.edu]|California Univer...|  Pennsylvania|[http://www.cup.e...|
|       [clarion.edu]|Clarion Universit...|  Pennsylvania|[http://www.clari...|
|  [allencollege.edu]|       Allen College|          Iowa|[https://www.alle...|
|           [esu.edu]|East Stroudsburg ...|  Pennsylvania|[http://www.esu.e...|
|      [edinboro.e

In [7]:
"""Count the number of universities that appeard in the list per state"""
state_sum = df.groupBy("state-province").count()
state_sum.show(50)

+--------------+-----+
|state-province|count|
+--------------+-----+
|          Utah|    1|
|         Texas|    4|
|  Pennsylvania|   31|
|            NV|    1|
|            VA|    1|
|          Iowa|    1|
|            TX|    2|
|       Indiana|    1|
|          Ohio|    2|
|            IN|    1|
|      New York|    1|
|      Colorado|    1|
|            CA|    2|
|      Illinois|    1|
|North Carolina|    2|
| New Hampshire|    1|
|South Carolina|    1|
|    Washington|    2|
|      Missouri|    1|
|      Virginia|    1|
|    California|    5|
|            NY|    9|
|            MI|    1|
|            ND|    1|
|      Michigan|    1|
|  New York, NY|    4|
|            GA|    2|
|       Florida|    1|
|         Maine|    1|
+--------------+-----+



In [8]:
"""University Name Contains the name of the state"""
state_univers = df.filter(col("name").contains(col("state-province")))
print('State Name is in College Name:',state_univers.count())
state_univers.select('state-province','name').show(state_univers.count(),truncate=False)

State Name is in College Name: 38
+--------------+-------------------------------------------------+
|state-province|name                                             |
+--------------+-------------------------------------------------+
|Texas         |The University of Texas at Rio Grande Valley     |
|Utah          |Utah Valley University                           |
|Pennsylvania  |Bloomsburg University of Pennsylvania            |
|Pennsylvania  |California University of Pennsylvania            |
|Pennsylvania  |Clarion University of Pennsylvania               |
|Pennsylvania  |East Stroudsburg State University of Pennsylvania|
|Pennsylvania  |Edinboro University of Pennsylvania              |
|Pennsylvania  |Haverford College in Pennsylvania                |
|Indiana       |Indiana University - Bloomington                 |
|Pennsylvania  |Indiana University of Pennsylvania               |
|Pennsylvania  |Kutztown University of Pennsylvania              |
|Pennsylvania  |Mansfield Un